In [1]:
from data import generer
import pandas as pd
import duckdb
import pathlib
import shutil
from loguru import logger

## Parquet

### Lage parquet-filer

In [2]:
mappe = pathlib.Path("data")
out_path = mappe / "data.parquet"

try:
    shutil.rmtree(mappe)
except FileNotFoundError as e:
    logger.info(e)
mappe.mkdir(exist_ok=True, parents=True)

2023-06-02 12:10:01.239 | INFO     | __main__:<module>:7 - [Errno 2] No such file or directory: 'data'


In [3]:
n = int(1e6)
df = generer(n)
df.head()

,id,tidspunkt,kategori
0,176de185-9796-4868-b935-a28d72038d03,2023-03-25 03:36:22.825494,inntektsmelding
1,029ae549-c070-4d92-88ee-a9d9e4fbe96e,2023-04-24 20:58:55.592941,inntektsmelding
2,fe93b7d8-d87e-45fb-b5fe-ac2dcaecd6d0,2023-02-20 14:41:09.939496,sykmelding
3,83cec294-56f8-4c5c-90d6-e2b840a14882,2023-02-14 04:29:18.575486,vedtak
4,19bf50f9-3656-4bf0-8e4f-3cbd43c7caad,2023-03-21 20:32:03.351933,søknad


In [5]:
df.dtypes

id                   object
tidspunkt    datetime64[ns]
kategori             object
dtype: object

In [6]:
df.tidspunkt.min(), df.tidspunkt.max()

(Timestamp('2023-01-01 00:00:02.181653'),
 Timestamp('2023-04-30 23:59:43.972925'))

In [7]:
df.to_parquet(out_path, engine='pyarrow')
df.to_csv(mappe / "data.csv", index=False)

In [8]:
!ls -lh data

total 251984
-rw-r--r--  1 eirik  staff    71M Jun  2 12:12 data.csv
-rw-r--r--  1 eirik  staff    43M Jun  2 12:12 data.parquet


In [17]:
df = pd.read_parquet(out_path, engine="pyarrow")#, dtype_backend="pyarrow")

In [14]:
df.head()

,id,tidspunkt,kategori
0,176de185-9796-4868-b935-a28d72038d03,2023-03-25 03:36:22.825494,inntektsmelding
1,029ae549-c070-4d92-88ee-a9d9e4fbe96e,2023-04-24 20:58:55.592941,inntektsmelding
2,fe93b7d8-d87e-45fb-b5fe-ac2dcaecd6d0,2023-02-20 14:41:09.939496,sykmelding
3,83cec294-56f8-4c5c-90d6-e2b840a14882,2023-02-14 04:29:18.575486,vedtak
4,19bf50f9-3656-4bf0-8e4f-3cbd43c7caad,2023-03-21 20:32:03.351933,søknad


In [16]:
df.memory_usage(deep=True).sum() // 1e6

61.0

In [21]:
df.dtypes

id                   object
tidspunkt    datetime64[ns]
kategori             object
dtype: object

In [31]:
df = pd.read_csv(mappe / 'data.csv')

In [35]:
df.memory_usage(deep=True).sum() / 1e6

248.25492

### Hent litt data fra en parquet-fil

In [36]:
in_path = mappe / "data.parquet"
pd.read_parquet(in_path).query("kategori == 'vedtak'")[['tidspunkt']].head()

,tidspunkt
3,2023-02-14 04:29:18.575486
8,2023-01-14 12:47:43.260776
11,2023-02-21 18:48:43.604952
24,2023-04-10 14:11:27.561978
26,2023-02-12 21:38:31.471382


In [37]:
pd.read_parquet(in_path, columns=['tidspunkt'], filters=[('kategori', '=', 'vedtak')])

,tidspunkt
0,2023-02-14 04:29:18.575486
1,2023-01-14 12:47:43.260776
2,2023-02-21 18:48:43.604952
3,2023-04-10 14:11:27.561978
4,2023-02-12 21:38:31.471382
...,...
250278,2023-04-29 20:26:00.851863
250279,2023-01-30 14:55:37.002569
250280,2023-04-02 16:39:44.792430
250281,2023-04-09 13:19:56.100086


### Hente litt data fra flere parquet-filer

In [38]:
!cp data/data.parquet data/data2.parquet
!cp data/data.parquet data/data3.parquet

In [39]:
!ls -lah data

total 426560
drwxr-xr-x   6 eirik  staff   192B Jun  2 12:22 .
drwxr-xr-x  10 eirik  staff   320B Jun  2 12:10 ..
-rw-r--r--   1 eirik  staff    71M Jun  2 12:12 data.csv
-rw-r--r--   1 eirik  staff    43M Jun  2 12:12 data.parquet
-rw-r--r--   1 eirik  staff    43M Jun  2 12:22 data2.parquet
-rw-r--r--   1 eirik  staff    43M Jun  2 12:22 data3.parquet


In [40]:
paths = mappe.glob('*.parquet')
# for path in g:
#     print(path)

In [41]:
dfs = (pd.read_parquet(path=path, engine='pyarrow') for path in paths)
# for df in gdf:
#     print(len(df))

In [42]:
pd.concat(dfs)

,id,tidspunkt,kategori
0,176de185-9796-4868-b935-a28d72038d03,2023-03-25 03:36:22.825494,inntektsmelding
1,029ae549-c070-4d92-88ee-a9d9e4fbe96e,2023-04-24 20:58:55.592941,inntektsmelding
2,fe93b7d8-d87e-45fb-b5fe-ac2dcaecd6d0,2023-02-20 14:41:09.939496,sykmelding
3,83cec294-56f8-4c5c-90d6-e2b840a14882,2023-02-14 04:29:18.575486,vedtak
4,19bf50f9-3656-4bf0-8e4f-3cbd43c7caad,2023-03-21 20:32:03.351933,søknad
...,...,...,...
999995,9a2ece21-8c22-4a12-aee1-d8c210eef89e,2023-04-09 13:19:56.100086,vedtak
999996,c773bdaa-6c63-4ddd-8205-4c309f716323,2023-02-03 03:07:13.921708,vedtak
999997,b085f7cc-fbe7-4c91-a4ea-1691bed66b68,2023-01-12 10:35:29.100118,inntektsmelding
999998,aaf45869-c11e-4d95-9cc0-5ca085f4eae1,2023-03-16 17:27:18.396483,sykmelding


## DuckDB

### Les parquet-filer

In [43]:
!ls -l data/data*.parquet

-rw-r--r--  1 eirik  staff  44687939 Jun  2 12:12 data/data.parquet
-rw-r--r--  1 eirik  staff  44687939 Jun  2 12:22 data/data2.parquet
-rw-r--r--  1 eirik  staff  44687939 Jun  2 12:22 data/data3.parquet


In [48]:
duckdb.query("""--sql
select id, tidspunkt as antall from 'data/data*.parquet'
where tidspunkt between '2023-03-01 01:00' and '2023-03-01 02:00'
and kategori = 'vedtak'
""").to_df()

,id,antall
0,ac740318-448f-43a8-bc4c-fe150939da6f,2023-03-01 01:03:16.641822
1,55834498-12c0-42fd-997f-2d3f6d53c9b9,2023-03-01 01:41:12.442596
2,418be0c4-6eef-4d88-a2a8-c0731aee51f6,2023-03-01 01:04:33.384429
3,fb72b3f3-a622-4dbf-ba77-d002f0a487e2,2023-03-01 01:15:43.574386
4,792d1788-b038-4fcb-b33e-7cc63757309d,2023-03-01 01:11:59.595969
...,...,...
241,8fe59ae3-9b90-4fbe-9762-db0c110dabdf,2023-03-01 01:17:08.058763
242,248ec8b5-3f35-4bde-bb25-05d246dcb365,2023-03-01 01:52:15.654553
243,aa4c614e-3db1-4ea6-9bd2-dd82adb3d5ce,2023-03-01 01:12:48.248956
244,144ce313-1b49-47e8-9f08-9feee06c2fc2,2023-03-01 01:04:06.690095


### Les mange parquet-filer

In [49]:
!./kopier.zsh

+./kopier.zsh:2> i=1
+./kopier.zsh:4> newpath=data/data_1.parquet 
+./kopier.zsh:5> cp data/data.parquet data/data_1.parquet
+./kopier.zsh:2> i=2
+./kopier.zsh:4> newpath=data/data_2.parquet 
+./kopier.zsh:5> cp data/data.parquet data/data_2.parquet
+./kopier.zsh:2> i=3
+./kopier.zsh:4> newpath=data/data_3.parquet 
+./kopier.zsh:5> cp data/data.parquet data/data_3.parquet
+./kopier.zsh:2> i=4
+./kopier.zsh:4> newpath=data/data_4.parquet 
+./kopier.zsh:5> cp data/data.parquet data/data_4.parquet
+./kopier.zsh:2> i=5
+./kopier.zsh:4> newpath=data/data_5.parquet 
+./kopier.zsh:5> cp data/data.parquet data/data_5.parquet
+./kopier.zsh:2> i=6
+./kopier.zsh:4> newpath=data/data_6.parquet 
+./kopier.zsh:5> cp data/data.parquet data/data_6.parquet
+./kopier.zsh:2> i=7
+./kopier.zsh:4> newpath=data/data_7.parquet 
+./kopier.zsh:5> cp data/data.parquet data/data_7.parquet
+./kopier.zsh:2> i=8
+./kopier.zsh:4> newpath=data/data_8.parquet 
+./kopier.zsh:5> cp data/data.parquet data/data_8.parquet


In [50]:
!du -hs data

 42G	data


In [51]:
paths = mappe.glob('data_*.parquet')
dfs = (pd.read_parquet(path) for path in paths)
# sum((1 for _ in paths))

In [52]:
antall_rader = 0
from tqdm.auto import tqdm
for df in tqdm(dfs, total=1000):
    antall_rader += len(df)
antall_rader

/Users/eirik/.pyenv/versions/3.11.3/envs/duckdb/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 21%|██▏       | 214/1000 [00:45<02:47,  4.69it/s]


KeyboardInterrupt: 

In [ ]:
# duckdb.query("""--sql
# COPY (SELECT *, tidspunkt::date dato FROM 'data/data*.parquet')
# TO 'data/partisjonert'
# (FORMAT PARQUET, PARTITION_BY (dato));
# """)

In [62]:
df = duckdb.query("""--sql
select * from 'data/data*.parquet'
where kategori in ('søknad', 'vedtak')
and tidspunkt::date = '2023-03-01'
order by tidspunkt desc
""").to_df()

In [ ]:

# for kategori in ['inntektsmelding', 'vedtak', 'søknad', 'sykmelding']:
#     con.query(f"""--sql
#     CREATE TABLE {kategori} AS
#     SELECT * FROM 'data/data.parquet'
#     WHERE kategori = '{kategori}'
#     """)
# con.query("select * from inntektsmelding limit 1")

### Transformer parquet-filer

In [66]:
in_path = mappe / "data*.parquet"
out_path = mappe / "daglig.parquet"
out_path.unlink(missing_ok=True)

In [67]:
duckdb.query(f"""--sql
COPY (
select
    tidspunkt::date dato,
    kategori,
    count(1) antall,
from '{in_path}'
where tidspunkt between '2023-03-01' and '2023-04-01'
group by kategori, dato
order by dato
)
TO '{out_path}'
""")

In [70]:
df = pd.read_parquet(out_path)
df

,dato,kategori,antall
0,2023-03-01,sykmelding,2099279
1,2023-03-01,søknad,2108306
2,2023-03-01,inntektsmelding,2126360
3,2023-03-01,vedtak,2043111
4,2023-03-02,sykmelding,2084234
...,...,...,...
119,2023-03-30,vedtak,2012018
120,2023-03-31,inntektsmelding,2158456
121,2023-03-31,søknad,2152438
122,2023-03-31,sykmelding,2079219


In [68]:
duckdb.query(f"""--sql
SELECT * FROM '{out_path}'
""")

┌────────────┬─────────────────┬─────────┐
│    dato    │    kategori     │ antall  │
│    date    │     varchar     │  int64  │
├────────────┼─────────────────┼─────────┤
│ 2023-03-01 │ sykmelding      │ 2099279 │
│ 2023-03-01 │ søknad          │ 2108306 │
│ 2023-03-01 │ inntektsmelding │ 2126360 │
│ 2023-03-01 │ vedtak          │ 2043111 │
│ 2023-03-02 │ sykmelding      │ 2084234 │
│ 2023-03-02 │ inntektsmelding │ 2196570 │
│ 2023-03-02 │ søknad          │ 2074204 │
│ 2023-03-02 │ vedtak          │ 2164474 │
│ 2023-03-03 │ sykmelding      │ 2050132 │
│ 2023-03-03 │ søknad          │ 2082228 │
│     ·      │   ·             │    ·    │
│     ·      │   ·             │    ·    │
│     ·      │   ·             │    ·    │
│ 2023-03-29 │ vedtak          │ 2086240 │
│ 2023-03-29 │ søknad          │ 2081225 │
│ 2023-03-30 │ sykmelding      │ 1975910 │
│ 2023-03-30 │ inntektsmelding │ 2035087 │
│ 2023-03-30 │ søknad          │ 2112318 │
│ 2023-03-30 │ vedtak          │ 2012018 │
│ 2023-03-3